# Reproduction of Adult dataset experiments

In this notebook we reproduce the results from Table 2 of the DECAF paper. We compare various methods for generating debiased data using the DECAF model against synthetic data generated using benchmark models GAN, WGAN-GP and FairGAN. As described in the paper we run all experiments (as implemented in this notebook) 10 times and avarage the results.

In [1]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

from data import load_adult, preprocess_adult
from metrics import DP, FTU
from train import train_decaf, train_fairgan, train_vanilla_gan, train_wgan_gp
# import tensorflow as tf

import tensorflow as tf

print(tf.__version__)
tf.config.run_functions_eagerly(True)


/Users/alyssaunell/UvA_FACT2022/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.4.4


## Loading data

In [2]:
dataset = load_adult()
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


Preprocess the data next in order to make it suitable for training models on.

In [3]:
dataset = preprocess_adult(dataset)
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.301370,0.833333,0.043350,0.000000,0.800000,0.333333,0.615385,0.6,0.0,1.0,0.02174,0.0,0.397959,0.0,1.0
1,0.452055,0.166667,0.047274,0.000000,0.800000,0.000000,0.307692,0.4,0.0,1.0,0.00000,0.0,0.122449,0.0,1.0
2,0.287671,0.000000,0.136877,0.200000,0.533333,0.166667,0.461538,0.6,0.0,1.0,0.00000,0.0,0.397959,0.0,1.0
3,0.493151,0.000000,0.149792,0.133333,0.400000,0.000000,0.461538,0.4,1.0,1.0,0.00000,0.0,0.397959,0.0,1.0
4,0.150685,0.000000,0.219998,0.000000,0.800000,0.000000,0.384615,0.0,1.0,0.0,0.00000,0.0,0.397959,0.3,1.0


Split the dataset into train and test folds. Test fold size is 2000.

In [34]:
datasetMat=dataset.corr()
dataSeries = datasetMat.unstack().sort_values()
print(dataSeries)
for label, item in dataSeries:
    print(label, item)


marital-status  sex              -0.381525
sex             marital-status   -0.381525
education-num   income           -0.332800
income          education-num    -0.332800
                age              -0.237040
                                    ...   
education       education         1.000000
fnlwgt          fnlwgt            1.000000
workclass       workclass         1.000000
native-country  native-country    1.000000
income          income            1.000000
Length: 225, dtype: float64


TypeError: cannot unpack non-iterable float object

In [4]:
# Split data into train and testing sets
dataset_train, dataset_test = train_test_split(dataset, test_size=2000,
                                               stratify=dataset['income'])

print('Size of train set:', len(dataset_train))
print('Size of test set:', len(dataset_test))

Size of train set: 43222
Size of test set: 2000


### Defining the DAG

We need to define a DAG which captures the biases of the dataset. As described in the DECAF paper normally a causal discovery algorithm is used. In this notebook we simply copy the DAG which as described in the Zhang et al. paper which is the one also used in the DECAF paper.

In [20]:
# Define DAG for Adult dataset
dag = [
    # Edges from race
    ['race', 'occupation'],
    ['race', 'income'],
    ['race', 'hours-per-week'],
    ['race', 'education'],
    ['race', 'marital-status'],

    # Edges from age
    ['age', 'occupation'],
    ['age', 'hours-per-week'],
    ['age', 'income'],
    ['age', 'workclass'],
    ['age', 'marital-status'],
    ['age', 'education'],
    ['age', 'relationship'],
    
    # Edges from sex
    ['sex', 'occupation'],
    ['sex', 'marital-status'],
    ['sex', 'income'],
    ['sex', 'workclass'],
    ['sex', 'education'],
    ['sex', 'relationship'],
    
    # Edges from native country
    ['native-country', 'marital-status'],
    ['native-country', 'hours-per-week'],
    ['native-country', 'education'],
    ['native-country', 'workclass'],
    ['native-country', 'income'],
    ['native-country', 'relationship'],
    
    # Edges from marital status
    ['marital-status', 'occupation'],
    ['marital-status', 'hours-per-week'],
    ['marital-status', 'income'],
    ['marital-status', 'workclass'],
    ['marital-status', 'relationship'],
    ['marital-status', 'education'],
    
    # Edges from education
    ['education', 'occupation'],
    ['education', 'hours-per-week'],
    ['education', 'income'],
    ['education', 'workclass'],
    ['education', 'relationship'],
    
    # All remaining edges
    ['occupation', 'income'],
    ['hours-per-week', 'income'],
    ['workclass', 'income'],
    ['relationship', 'income'],

    # #added edges
    ['income', 'education']
]

def dag_to_idx(df, dag):
    """Convert columns in a DAG to the corresponding indices."""

    dag_idx = []
    for edge in dag:
        dag_idx.append([df.columns.get_loc(edge[0]), df.columns.get_loc(edge[1])])

    return dag_idx

# Convert the DAG to one that can be provided to the DECAF model
dag_seed = dag_to_idx(dataset, dag)
print(dag_seed)

[[8, 6], [8, 14], [8, 12], [8, 3], [8, 5], [0, 6], [0, 12], [0, 14], [0, 1], [0, 5], [0, 3], [0, 7], [9, 6], [9, 5], [9, 14], [9, 1], [9, 3], [9, 7], [13, 5], [13, 12], [13, 3], [13, 1], [13, 14], [13, 7], [5, 6], [5, 12], [5, 14], [5, 1], [5, 7], [5, 3], [3, 6], [3, 12], [3, 14], [3, 1], [3, 7], [6, 14], [12, 14], [1, 14], [7, 14], [14, 3]]


It's also necessary to define edges we want to remove from the DAG in order to meet the various fairness criteria described in the paper.

In [23]:
def create_bias_dict(df, edge_map):
    """
    Convert the given edge tuples to a bias dict used for generating
    debiased synthetic data.
    """
    bias_dict = {}
    for key, val in edge_map.items():
        bias_dict[df.columns.get_loc(key)] = [df.columns.get_loc(f) for f in val]
    
    return bias_dict

# Bias dictionary to satisfy FTU
bias_dict_ftu = create_bias_dict(dataset, {'income': ['sex']})
print('Bias dict FTU:', bias_dict_ftu)

# Bias dictionary to satisfy DP
bias_dict_dp = create_bias_dict(dataset, {'income': [
    'occupation', 'hours-per-week', 'marital-status', 'education', 'sex',
    'workclass', 'relationship'], 'education':['income']})
print('Bias dict DP:', bias_dict_dp)

# Bias dictionary to satisfy CF
bias_dict_cf = create_bias_dict(dataset, {'income': [
    'marital-status', 'sex']})
print('Bias dict CF:', bias_dict_cf)

Bias dict FTU: {14: [9]}
Bias dict DP: {14: [6, 12, 5, 3, 9, 1, 7], 3: [14]}
Bias dict CF: {14: [5, 9]}


## Experiments

We have loaded and preprocessed the data and we are ready to run the experiments. For each experiment we train a generative model, sample synthetic data from the trained model and then obtain metrics by training and evaluating a downstream multi-layer perceptron using the test fold we generated in the previous section. We use the MLP model from `sklearn` with default parameters which matches the settings described in Appendix D of the paper.

In [24]:
def eval_model(dataset_train, dataset_test):
    """Helper function that prints evaluation metrics."""

    X_train, y_train = dataset_train.drop(columns=['income']), dataset_train['income']
    X_test, y_test = dataset_test.drop(columns=['income']), dataset_test['income']

    clf = MLPClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auroc = roc_auc_score(y_test, y_pred)
    dp = DP(clf, X_test)
    ftu = FTU(clf, X_test)

    return {'precision': precision, 'recall': recall, 'auroc': auroc,
            'dp': dp, 'ftu': ftu}

### Original dataset

As a benchmark we want to first train the downstream model on the original dataset.

In [25]:
eval_model(dataset_train, dataset_test)

/Users/alyssaunell/UvA_FACT2022/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.8721147847785402,
 'recall': 0.9295212765957447,
 'auroc': 0.7581074124914208,
 'dp': 0.1487722530238096,
 'ftu': 0.02949999999999997}

In the following sections we train various models in order to reproduce the results from Table 2 of the DECAF paper.

### GAN

In [11]:
synth_data = train_vanilla_gan(dataset_train)
synth_data.head()

Synthetic data generation: 100%|██████████| 338/338 [00:05<00:00, 65.94it/s]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.054608,0.166667,0.005505,0.666667,0.333333,0.000000,0.076923,0.6,0.50,1.0,-0.01159,0.010739,0.003773,0.250,0.0
1,0.054608,0.500000,0.005505,0.933333,0.333333,0.333333,0.692308,0.0,1.00,1.0,-0.01159,0.010739,0.003773,0.450,1.0
2,0.054608,0.666667,0.005505,0.133333,1.000000,0.166667,1.000000,0.0,0.75,1.0,-0.01159,0.010739,0.003773,1.000,0.0
3,0.054608,0.500000,0.005505,0.533333,0.866667,0.833333,0.076923,0.8,0.00,1.0,-0.01159,0.010739,0.003773,0.025,1.0
4,0.054608,0.666667,0.005505,0.666667,0.600000,0.500000,0.076923,1.0,1.00,0.0,-0.01159,0.010739,0.003773,0.750,1.0


In [18]:
eval_model(synth_data, dataset_test)

{'precision': 0.9100719424460432,
 'recall': 0.16821808510638298,
 'auroc': 0.5589074296499656,
 'dp': 0.027871055971599723,
 'ftu': 0.096}

### WGAN-GP

In [11]:
synth_data = train_wgan_gp(dataset_train)
synth_data.head()

Synthetic data generation: 100%|██████████| 87/87 [00:02<00:00, 36.15it/s]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.653928,0.000000,0.121905,0.000000,0.800000,0.166667,0.307692,0.4,0.50,1.0,0.056760,0.031365,0.606606,0.425,0.0
1,0.501356,0.333333,0.122377,0.600000,0.066667,0.833333,0.076923,1.0,0.25,0.0,-0.002472,-0.038353,0.504046,0.150,0.0
2,0.617011,0.000000,0.123270,0.466667,0.600000,0.666667,0.384615,0.4,0.00,1.0,0.054731,0.043257,0.588315,0.575,1.0
3,0.468021,0.000000,0.112826,1.000000,0.000000,0.000000,0.076923,0.6,0.75,0.0,0.005098,-0.020421,0.456802,0.825,0.0
4,0.484573,0.166667,0.111068,0.266667,0.200000,0.000000,0.769231,0.2,0.25,1.0,0.044819,0.015543,0.481428,0.150,1.0


In [12]:
eval_model(synth_data, dataset_test)

/Users/alyssaunell/UvA_FACT2022/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


{'precision': 0.883248730964467,
 'recall': 0.3470744680851064,
 'auroc': 0.6039807824296499,
 'dp': 0.3568324130405257,
 'ftu': 0.15100000000000002}

### FairGAN

In [ ]:
synth_data = train_fairgan(dataset_train)
synth_data.head()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
cache/adult.npy


2022-12-05 12:39:15.329775: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)


Pretrain_Epoch:0, trainLoss:0.022373, validLoss:0.011082, validReverseLoss:0.000000
Pretrain_Epoch:1, trainLoss:0.010851, validLoss:0.011148, validReverseLoss:0.000000
Pretrain_Epoch:2, trainLoss:0.010906, validLoss:0.011166, validReverseLoss:0.000000
Pretrain_Epoch:3, trainLoss:0.008346, validLoss:0.003728, validReverseLoss:0.000000
Pretrain_Epoch:4, trainLoss:0.003643, validLoss:0.003653, validReverseLoss:0.000000
Pretrain_Epoch:5, trainLoss:0.003606, validLoss:0.003679, validReverseLoss:0.000000
Pretrain_Epoch:6, trainLoss:0.003586, validLoss:0.003621, validReverseLoss:0.000000
Pretrain_Epoch:7, trainLoss:0.003168, validLoss:0.002621, validReverseLoss:0.000000
Pretrain_Epoch:8, trainLoss:0.002528, validLoss:0.002654, validReverseLoss:0.000000
Pretrain_Epoch:9, trainLoss:0.002522, validLoss:0.002629, validReverseLoss:0.000000
Pretrain_Epoch:10, trainLoss:0.002509, validLoss:0.002633, validReverseLoss:0.000000
Pretrain_Epoch:11, trainLoss:0.002512, validLoss:0.002623, validReverseLoss

/Users/alyssaunell/UvA_FACT2022/venv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.000000,0.231094,0.552807,0.411596,0.434765,0.149908,0.562647,0.155949,0.398961,0.0,0.700903,0.309791,0.00000,0.915657,0.0
1,0.000000,0.174592,0.000000,0.563940,0.260049,0.597330,1.091372,0.000000,0.897431,0.0,0.000000,0.471334,0.00000,1.061039,1.0
2,0.000000,0.150367,0.000000,0.513259,0.000000,0.006482,0.600942,0.000000,1.091244,0.0,0.000000,0.573798,0.00000,1.059351,1.0
3,1.071063,0.169032,0.460340,0.559052,0.332120,0.417909,0.476185,0.223396,0.403090,0.0,0.000000,0.423902,0.00000,0.954979,1.0
4,0.000000,0.133058,0.401199,0.553105,0.063670,0.562526,0.804348,0.000000,0.050657,0.0,0.000000,0.646138,1.02699,0.000000,1.0


In [ ]:
eval_model(synth_data, dataset_test)

{'precision': 0.7874396135265701,
 'recall': 0.9753989361702128,
 'auroc': 0.5885059196980096,
 'dp': 0.06786115049964703,
 'ftu': 0.045499999999999985}

### DECAF

#### DECAF-ND

In [26]:
synth_data = train_decaf(dataset_train, dag_seed)
synth_data.head()

/Users/alyssaunell/UvA_FACT2022/venv/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/alyssaunell/UvA_FACT2022/venv/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
20452,0.193666,2.667125e-05,0.117253,0.207648,0.618560,0.289106,0.062816,0.533941,1.356863e-06,1.0,8.721382e-14,2.093533e-16,0.483328,6.928600e-22,1.0
16182,0.307964,7.430633e-01,0.126537,0.189040,0.624500,0.016542,0.228374,0.397712,4.784805e-37,1.0,4.526148e-05,3.411813e-18,0.736634,9.945138e-17,0.0
11871,0.339429,8.056119e-07,0.196003,0.037737,0.619057,0.273228,0.732520,0.920769,1.390855e-28,0.0,3.400319e-15,1.602074e-06,0.640687,4.796973e-16,1.0
16265,0.231148,1.223689e-06,0.225775,0.043755,0.591762,0.274211,0.221933,0.054984,9.931018e-01,0.0,5.596818e-07,7.191827e-17,0.266740,5.959776e-25,0.0
11031,0.232253,1.954066e-07,0.153415,0.046132,0.569815,0.497894,0.509483,0.513637,1.150797e-19,0.0,2.716498e-08,3.339329e-05,0.308749,2.210476e-25,1.0


In [27]:
eval_model(synth_data, dataset_test)

{'precision': 0.8640059127864006,
 'recall': 0.7772606382978723,
 'auroc': 0.7031464481811942,
 'dp': 0.3107882,
 'ftu': 0.167}

#### DECAF-FTU

In [14]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_ftu)
synth_data.head()

/Users/alyssaunell/UvA_FACT2022/venv/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/alyssaunell/UvA_FACT2022/venv/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
20452,0.382177,1.950229e-07,0.066027,0.091434,0.791150,3.444650e-01,0.329121,0.194112,8.084046e-13,0.0,2.419608e-16,4.068636e-03,0.370711,9.692002e-23,1.0
16182,0.074729,1.842130e-03,0.215484,0.037902,0.807863,2.961568e-01,0.267855,0.179010,2.100054e-36,0.0,3.771333e-12,4.393299e-07,0.290560,7.465259e-25,1.0
11871,0.099151,2.670872e-02,0.203023,0.098252,0.613096,9.864419e-06,0.266112,0.420981,1.875740e-06,1.0,4.098765e-09,4.076800e-04,0.421974,1.553486e-17,1.0
16265,0.092774,4.392007e-03,0.118775,0.082336,0.607450,4.803772e-01,0.422896,0.299001,9.703801e-01,1.0,8.716559e-04,3.341999e-12,0.462055,2.307131e-25,1.0
11031,0.084986,8.045063e-06,0.054866,0.096308,0.652466,2.549023e-07,0.212599,0.411495,7.559609e-30,1.0,4.642411e-20,9.850017e-06,0.393950,5.852425e-17,1.0


In [15]:
eval_model(synth_data, dataset_test)

{'precision': 0.8367071524966262,
 'recall': 0.824468085106383,
 'auroc': 0.6682824296499656,
 'dp': 0.15891743,
 'ftu': 0.043000042}

#### DECAF-CF

In [16]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_cf)
synth_data.head()

/Users/alyssaunell/UvA_FACT2022/venv/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/alyssaunell/UvA_FACT2022/venv/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
20452,0.133737,1.189730e-07,0.126295,0.105701,0.477695,2.580135e-01,0.627097,0.366582,4.188384e-13,0.0,1.002909e-19,2.406454e-16,0.243499,5.296614e-24,1.0
16182,0.093253,1.436526e-02,0.135612,0.024180,0.615963,2.706627e-01,0.235935,0.283575,3.590776e-33,0.0,6.304817e-12,1.031040e-14,0.284899,7.314462e-14,1.0
11871,0.093267,9.099595e-05,0.106994,0.202505,0.618928,1.894813e-01,0.203234,0.268246,2.853473e-10,1.0,1.130472e-15,4.292134e-21,0.601092,2.356547e-14,0.0
16265,0.259172,3.368842e-02,0.263538,0.206944,0.477980,2.639616e-01,0.165897,0.673376,1.876876e-22,0.0,3.565310e-16,1.966370e-23,0.470767,5.659096e-09,1.0
11031,0.404082,7.223784e-01,0.166777,0.112113,0.623372,6.047108e-07,0.491174,0.398655,3.313650e-32,1.0,5.420171e-17,4.584836e-12,0.606068,5.016220e-11,1.0


In [17]:
eval_model(synth_data, dataset_test)

{'precision': 0.7667025862068966,
 'recall': 0.9461436170212766,
 'auroc': 0.5365798730267674,
 'dp': 0.015412927,
 'ftu': 0.048000038}

#### DECAF-DP

In [18]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_dp)
synth_data.head()

/Users/alyssaunell/UvA_FACT2022/venv/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/Users/alyssaunell/UvA_FACT2022/venv/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
20452,0.083235,1.378219e-02,0.143117,0.020544,0.903827,7.596659e-07,0.426143,0.421437,3.127489e-33,1.0,1.371107e-11,6.240011e-23,0.548103,2.787938e-04,0.0
16182,0.351434,3.748482e-03,0.073219,0.161803,0.986350,2.861480e-01,0.262494,0.422429,7.944885e-01,0.0,1.886600e-16,2.844837e-11,0.420165,4.087153e-22,0.0
11871,0.246005,7.532038e-01,0.073301,0.055454,0.576468,1.435577e-03,0.366864,0.404960,4.953091e-34,1.0,8.436563e-06,9.436003e-17,0.412014,1.789019e-14,1.0
16265,0.217287,4.170377e-06,0.056176,0.219164,0.631239,1.443454e-01,0.480455,0.244106,4.291821e-37,1.0,6.497776e-15,1.197551e-23,0.288865,1.493226e-09,1.0
11031,0.412754,2.749107e-07,0.055045,0.169982,0.598617,3.988742e-12,0.546007,0.392861,0.000000e+00,1.0,2.322783e-04,3.120778e-08,0.760667,3.835861e-24,0.0


In [19]:
eval_model(synth_data, dataset_test)

{'precision': 0.759709994821336,
 'recall': 0.9753989361702128,
 'auroc': 0.5199575326012355,
 'dp': 0.025110424,
 'ftu': 0.013000011}